<a href="https://colab.research.google.com/github/berry2971/senti-dict-petitions/blob/master/senti_dict_petitions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###과제 의의
* '불만'이라는 특정한 감성을 추출해내기란 쉽지 않다. 하지만 국민청원 게시판은 '불만, 원망, 간청'이라는 감성이 전적으로 드러나 있는 게시판이다. 이에 국민청원 게시판의 텍스트 분석을 통해 '불만'의 감성을 가진 단어들을 상당히 추출할 수 있을 것으로 보인다.

###과제 수행 방법
#####스크래핑
Selenium을 사용하되, 별도의 코드로 실행함. (https://github.com/berry2971/scraper-petitions)

#####분석 대상
* 130641건의 국민청원을 대상으로 함.
* 보건복지:	6317
* 저출산/고령화대책:	960
* 행정:	6673
* 미래:	7589
* 안전/환경:	11460
* 성장동력:	3506
* 기타:	16800
* 일자리:	6884
* 외교/통일/국방:	9385
* 반려동물:	1481
* 교통/건축/국토:	8603
* 경제민주화:	6591
* 농산어촌:	603
* 육아/교육:	6833
*  정치개혁:	19518
* 인권/성평등:	10177
* 문화/예술/체육/언론:	7261

####형태소 분석
* 형태소 분석의 경우, SentencePiece 등 비지도 기반 형태소 분석기를 사용할 수도 있지만, 국민청원의 경우 꽤나 정제된 언어로 작성될 뿐만 아니라 욕설 등 연구 가치가 있지만 OOV가 발생할 가능성이 높은 어휘들은 대부분 게시판 관리자에 의해 제거된다. 따라서 Mecab을 사용하였다.

####KNU(케이앤유) 한국어 감성 사전(군산대학교) 사용
* https://github.com/park1200656/KnuSentiLex
* 박상민(군산대학교)·나철원(군산대학교)·최민성(군산대학교)·이다희(군산대학교)· 온병원(군산대학교)(2018), Bi-LSTM 기반의 한국어 감성사전 구축 방안
* KNU 한국어 감성 사전은 형용사뿐만 아니라 부사 등을 포함하고 있어 보다 풍부한 어휘를 지니고 있고, 도메인에서 비교적 자유롭다. 14843개의 어휘로 구성되어 있다.
* 기발표된 KNU 한국어 감성 사전의 부정 극성 단어를 기반으로 하여, 해당 단어와 유사한 맥락을 지니는 단어들을 국민청원 게시물에서 Word2Vec을 통해 형용사(VA)/부사(MAG)/동사(VV)를 추출해 낸다.
* NNG, NNP의 경우 감성이 아닌 해당 시기의 이슈와 트렌드를 반영하여 분석될 확률이 다분하여 추출 대상에서 제외한다.
* KNU 한국어 감성 사전의 각 단어들에 대해 추출이 진행되기 때문에 중복되는 단어가 발생하는데, 이 경우 가장 높은 코사인 유사도를 단어와 함께 저장한다.

In [0]:
##### Mount Google drive
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
##### Connect with db (raw text)
import sqlite3
con1 = sqlite3.connect('/content/gdrive/My Drive/senti/result_petitions.db')
cur1 = con1.cursor()

In [0]:
##### Connect with db (analyzed text)
con2 = sqlite3.connect('/content/gdrive/My Drive/senti/result_petitions_tagged.db')

##### EXECUTE FIRST TIME ONLY
#con2.execute('CREATE TABLE analysis (TAGGED BLOB, CATEGORY TEXT, AGREES INTEGER)')

In [0]:
########## EXECUTE FIRST TIME ONLY ##########
##### Prepare pos tagger: mecab
!pip install -v python-mecab-ko
import mecab
mecab = mecab.MeCab()

In [0]:
########## EXECUTE FIRST TIME ONLY ##########
##### Prepare sentence divider: kss(Korean Sentence Splitter by 박상길)
!pip install kss
import kss

'''
raw = "안녕하세요. 오늘은 날씨가 맑네요! 하늘도 푸르구요! 미세먼지만 없으면ㅠㅠ 좋을텐데ㅠㅠ"
kss.split_sentences(raw)
'''

'\nraw = "안녕하세요. 오늘은 날씨가 맑네요! 하늘도 푸르구요! 미세먼지만 없으면ㅠㅠ 좋을텐데ㅠㅠ"\nkss.split_sentences(raw)\n'

In [0]:
########## SPENDS LONG TIME ##########
########## EXECUTE FIRST TIME ONLY ##########
##### Pos tag and save it into database
import pickle

count = 0
for row in cur1.execute("SELECT title, content, category, agrees FROM petitions"):
    # Insert pickled data into database
    tagged_sentences = [mecab.pos(sentence) for sentence in kss.split_sentences(row[0]+" "+row[1])] # data
    data = tagged_sentences
    pdata = pickle.dumps(data)
    con2.execute('INSERT INTO analysis (TAGGED, CATEGORY, AGREES) VALUES (?, ?, ?)', [pdata, row[2], row[3]])
  
    # Check progress and Commit database
    count += 1
    if count % 1000 == 0:
        print(count)
        con2.commit()

con2.commit()

categories:

{'보건복지', '저출산/고령화대책', '행정', '미래', '안전/환경', '성장동력', '기타', '일자리', '외교/통일/국방', '반려동물', '교통/건축/국토', '경제민주화', '농산어촌', '육아/교육', '정치개혁', '인권/성평등', '문화/예술/체육/언론'}

In [88]:
# Load database
import pickle

con3 = con2
cur3 = con3.cursor()

data_ctg = dict()
categories = ['보건복지', '저출산/고령화대책', '행정', '미래', '안전/환경', '성장동력', '기타', '일자리', '외교/통일/국방', '반려동물', '교통/건축/국토', '경제민주화', '농산어촌', '육아/교육', '정치개혁', '인권/성평등', '문화/예술/체육/언론']
count_doc_categories = []
for i in range(len(categories)):
    count_doc_categories.append(0)
for c in categories:
    data_ctg.update({c:[]})

count = 0
for docs in cur3.execute("SELECT * FROM analysis"):
    # Check progress
    count += 1
    if count % 15000 == 0:
        print(count, end='...')
        print(int(count/130641*100), end='')
        print("%")
    if count == 130641:
        print("130641...100%")

    content = pickle.loads(docs[0])
    category = docs[1]
    agrees = docs[2]

    # Count numbers
    count_doc_categories[categories.index(category)] += 1

    # Convert (안녕, IC) to 안녕/IC
    for sentence in content:
        for i in range(len(sentence)):
            word = sentence[i][0]
            pos = sentence[i][1]
            tagged_word = word+"/"+pos
            sentence[i] = tagged_word

    data_ctg[category].append((content, agrees))

for c in categories:
    print("{c}:\t{num}".format(c=c, num=count_doc_categories[categories.index(c)]))

15000...11%
30000...22%
45000...34%
60000...45%
75000...57%
90000...68%
105000...80%
120000...91%
130641...100%
보건복지:	6317
저출산/고령화대책:	960
행정:	6673
미래:	7589
안전/환경:	11460
성장동력:	3506
기타:	16800
일자리:	6884
외교/통일/국방:	9385
반려동물:	1481
교통/건축/국토:	8603
경제민주화:	6591
농산어촌:	603
육아/교육:	6833
정치개혁:	19518
인권/성평등:	10177
문화/예술/체육/언론:	7261


In [0]:
##### Preprocess data
data_ctg_sentences = dict() # {'보건복지':[['먼저/MAG','가/VV', '아/EF'], ['오늘/NNG', '도/JX']], '일자리':[['성공/NNG', '하/XSV', '았/EP', '나/EF'], ['글쎄/IC']], ...}
categories = ['보건복지', '저출산/고령화대책', '행정', '미래', '안전/환경', '성장동력', '기타', '일자리', '외교/통일/국방', '반려동물', '교통/건축/국토', '경제민주화', '농산어촌', '육아/교육', '정치개혁', '인권/성평등', '문화/예술/체육/언론']
for c in categories:
    data_ctg_sentences.update({c:[]})

for key in data_ctg.keys():
    sentences = []
    for d in data_ctg[key]:
        for sentence in d[0]:
            sentences.append(sentence)
    data_ctg_sentences[key].extend(sentences)

In [0]:
from gensim.models import Word2Vec

models = []

count = 0
for v in data_ctg_sentences.values():
    count += 1
    print("{cntt}/17".format(cntt=count))
    models.append(Word2Vec(v, size=300, window=3, min_count=10, workers=4, sg=1, iter=50, sample=0.00001))

In [0]:
def parse_token(token):
    word = re.search("(.*)/(.*)", token).group(1)
    pos = re.search("(.*)/(.*)", token).group(2)
    return (word, pos)

def parse_tagged_word(word):
    lst = word.split(sep = "/")
    return (lst[0], lst[1])

In [31]:
##### t-SNE: t-Stochastic Neighbor Embedding
import re
import matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

obj_model = models[0]

mag_list = [w for w in list(obj_model.wv.vocab) if parse_token(w)[1]=='MAG']



matplotlib.rcParams["axes.unicode_minus"] = False

#vocab = obj_model.wv.vocab
vocab = mag_list
X = obj_model[vocab]

tsne = TSNE(n_components = 2)
X_tsne = tsne.fit_transform(X)

##### Show data frame
import pandas as pd
df = pd.DataFrame(X_tsne, index = vocab, columns = ["x", "y"])
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


,x,y
워낙/MAG,4.992065,2.061124
제대로/MAG,1.679194,2.639233
안/MAG,0.790547,1.923129
함께/MAG,2.235300,9.134355
벌써/MAG,-4.085002,9.280380


In [0]:
##### Install Nanum Fonts
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [0]:
##### Draw figure
%matplotlib inline

fig = plt.figure()
fig.set_size_inches(30,20)

ax = fig.add_subplot(1, 1, 1)
ax.scatter(df["x"], df["y"])

prop = fm.FontProperties(fname = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf")
for word, pos in list(df.iterrows()):
  ax.annotate(word, pos, fontsize = 12, fontproperties = prop)

plt.show()

In [0]:
##### Read KNU neg_pol_word
knu_negv_words = set()
with open('/content/gdrive/My Drive/senti/neg_pol_word.txt', 'r', encoding = 'utf8') as f:
    knu_negv_file = f.readlines()

one_ej = 0
for i in range(len(knu_negv_file)):
    item = knu_negv_file[i]
    knu_negv_file[i] = item.strip()
    item = knu_negv_file[i]
    if (len(item.split(sep=' ')) == 1): # apply for single ej item only
        one_ej += 1
        pos_res = mecab.pos(item)
        tagged_item = pos_res[0][0]+"/"+pos_res[0][1]
        knu_negv_words.add(tagged_item)
knu_negv_words = list(knu_negv_words) # ['방탕/NNG', '^-^;;/SY', '결핍/NNG', '짖궂/VA', '맛없/VA']

In [87]:
##### Get word set
for i in range(len(models)):
    print("model_{name}".format(name=categories[i]))
    cur_model = models[i]
    cur_negv_words = []
    for word in knu_negv_words:
        try:
            cur_sims = cur_model.wv.most_similar(word, topn = 10)
            for w in cur_sims:
                if parse_tagged_word(w[0])[1] in ['MAG','VA','VV']:
                    cur_negv_words.append(w)
        except:
            continue

    wordset = set()
    for w in cur_negv_words:
        wordset.add(w[0])
    
    worddict = dict()
    for w in wordset:
        worddict[w] = []
    
    for w in cur_negv_words:
        worddict[w[0]].append(w[1])
    
    for key in worddict.keys():
        worddict[key] = max(worddict[key])
    
    print(list(worddict.items())[:10])
    print(len(list(worddict.items())))

model_보건복지


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('월등히/MAG', 0.5061543583869934), ('외치/VV', 0.3822783827781677), ('되묻/VV', 0.5748921632766724), ('쳐다보/VV', 0.5293914079666138), ('흘러가/VV', 0.5267065167427063), ('지금/MAG', 0.3621225655078888), ('말리/VV', 0.5711488723754883), ('점점점/MAG', 0.7370374202728271), ('붙잡/VV', 0.4601231515407562), ('급히/MAG', 0.37684696912765503)]
439
model_저출산/고령화대책
[('월등히/MAG', 0.944807767868042), ('맨날/MAG', 0.9918994307518005), ('나누/VV', 0.976642370223999), ('매번/MAG', 0.9933590292930603), ('간절히/MAG', 0.9853604435920715), ('좀/MAG', 0.9791259765625), ('심지어/MAG', 0.9619573354721069), ('잘/MAG', 0.9791260957717896), ('누리/VV', 0.9830825328826904), ('갑자기/MAG', 0.9862367510795593)]
219
model_행정
[('어짜피/MAG', 0.49736422300338745), ('되묻/VV', 0.35155895352363586), ('추/VV', 0.3514246940612793), ('차/VA', 0.5480322241783142), ('붙잡/VV', 0.4528171718120575), ('손놓/VV', 0.49715811014175415), ('모조리/MAG', 0.47510021924972534), ('싸/VV', 0.44332456588745117), ('너무/MAG', 0.4117417335510254), ('몰려들/VV', 0.5648715496063232)]
398
model_미래